In [1]:
%matplotlib inline

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
import datetime

## Helper functions for graphical representations and further data analysis

#### Dual boxplots for 2 act tool streams

In [2]:
def act_boxplot( df, stream_a, stream_b, save = False, name = "Dual_boxplot"):
#     dpi = 600
#     fig, axarr = plt.subplots(1, 2, figsize=(8, 12), dpi=dpi)
    fig, axarr = plt.subplots(1, 2, figsize=(8, 12))
    bplot_L=sns.boxplot( y=df[stream_a],
                     width=0.5,
    #                  palette="colorblind",
                     ax=axarr[0],
                     color="g")
    bplot_L=sns.despine(offset=10, trim=True, ax=axarr[0])
    bplot_L=sns.stripplot(y=df[stream_a],
                       jitter=True, 
                       marker='o', 
                       alpha=0.01,
                       color='black',
                       ax=axarr[0])
    bplot_W=sns.boxplot( y=df[stream_b],
                     width=0.5,
    #                  palette="colorblind",
                     ax=axarr[1],
                     color="b")
    bplot_W=sns.despine(offset=10, trim=True, ax=axarr[1])
    bplot_W=sns.stripplot(y=df[stream_b],
                       jitter=True, 
                       marker='o', 
                       alpha=0.01,
                       color='black',
                       ax=axarr[1])
    d_time = datetime.datetime.now().strftime("%y-%m-%d-%H-%M")
    if save:
        folder = '.\Data\\'
        filename = folder + d_time + '_' + name + '_' + stream_a + '_' + stream_b + '.png'
        plt.savefig(filename)

#### Boxplot and histogram for 1 act tool stream

In [3]:
def act_hist_box(df, stream_a, **kwargs):

    arg_dict = {
        'save' : False,
        'name' : 'Boxplot_and_Histogram'
    }
    
    for key, value in arg_dict.items():
        if key in kwargs:
            arg_dict[key] = kwargs[key]
            
    # Cut the window in 2 parts
    f, (ax_box, ax_hist) = plt.subplots(2, sharex=True, gridspec_kw={"height_ratios": (.15, .85)})

    # Add a graph in each part
    sns.boxplot(df[stream_a], ax=ax_box)
    sns.distplot(df[stream_a], ax=ax_hist)

    # Remove x axis name for the boxplot
    text = ax_box.set(xlabel='')
    
    # save to file
    d_time = datetime.datetime.now().strftime("%y-%m-%d-%H-%M")
    if arg_dict['save']:
        folder = '.\Data\\'
        filename = folder + d_time + '_' + arg_dict['name'] + '_' + stream_a + '.png'
        plt.savefig(filename)

#### Line plot drawing to merging frequencies and stream data

In [4]:
def emc_line_plot(df_plot, y_a, stream_dict, label_dict, plot_title, y_a_label, **kwargs):
    
    if stream_dict['min']<=stream_dict['lsl']:
        y_str_lim_a = stream_dict['min'] - 50
    else:
        y_str_lim_a = stream_dict['lsl'] - 50
    if stream_dict['max']>=stream_dict['usl']:
        y_str_lim_b = stream_dict['max'] + 50
    else:
        y_str_lim_b = stream_dict['usl'] + 50
       
    arg_dict = {
        'save' : False,
        'name' : 'Emc_Line_Plot',
        'process_window' : (300, 60000),
    }

    for key, value in arg_dict.items():
        if key in kwargs:
            arg_dict[key] = kwargs[key]
    
    fig, ax = plt.subplots(figsize=(20, 10))
    ax.set(xlim=arg_dict['process_window'], ylim=(y_str_lim_a, y_str_lim_b), \
           xlabel='Time[ms]', ylabel=y_a_label, title=plot_title)
    ax.axhline(stream_dict['usl'], color="r", linestyle="--", label = 'USL')
    ax.axhline(stream_dict['lsl'], color="r", linestyle="--", label = 'LSL')
    ax2 = ax.twinx()
    ax2.set(ylim=(0, 1.1), ylabel='Freq[on/off]')
    ax2.grid(False)
    ax.plot( 'key', y_a, data=df_plot, marker='', color='skyblue', linewidth=2)
    ax2.plot('key', 'freq_mark', data=df_plot, marker='', color='olive', linewidth=2)    
    plt.style.use('seaborn-darkgrid')
    plt.rcParams['font.sans-serif'] = 'Arial'
    plt.rcParams['font.family'] = 'sans-serif'
    plt.rcParams['text.color'] = '#909090'
    plt.rcParams['axes.labelcolor']= '#909090'
    plt.rcParams['xtick.color'] = '#909090'
    plt.rcParams['ytick.color'] = '#909090'
    plt.rcParams['font.size']=14
    color_palette_list = ['#009ACD', '#ADD8E6', '#63D1F4', '#0EBFE9',   
                          '#C1F0F6', '#0099CC']
    
    # adding labbels to each frequency
    for key, value in label_dict.items():
        if value['key'] > arg_dict['process_window'][1]:
            break
        ax2.annotate(value['freq_label'],
            xy=(value['key'], 1), xycoords='data',
            xytext=(-100, -100), textcoords='offset points',
            arrowprops=dict(arrowstyle="->",
                            connectionstyle="arc,angleA=0,armA=40,angleB=-90,armB=10,rad=7"))
    plt.style.use('seaborn-darkgrid')
    ax.legend(frameon=False, loc='upper left', ncol=3)
    ax2.legend(frameon=False, loc='upper right', ncol=1)
    
    # save to file
    d_time = datetime.datetime.now().strftime("%y-%m-%d-%H-%M")
    if arg_dict['save']:
        folder = '.\Data\\'
        filename = folder + d_time + '_' + arg_dict['name'] + '_' + y_a_label + '.png'
        plt.savefig(filename)
    plt.show()

#### Emc results scatter plot

In [5]:
def emc_scatter_plot(df_plot, plot_title, stream_spec, **kwargs):
    
    arg_dict = {
        'save' : False,
        'name' : 'Emc_Scatter_Plot',
        'x_process_window' : (0, 100000),
        'y_process_window' : (120000, 300000)
    }
    
    for key, value in arg_dict.items():
        if key in kwargs:
            arg_dict[key] = kwargs[key]
    
    fig, ax = plt.subplots(figsize=(20, 10))
    
    df_nok = df_plot[(df_plot['freq']!=0) & (df_plot[stream_spec]==0)].drop_duplicates(['freq'], keep='first')
    df_ok = df_plot[(df_plot['freq']!=0) & (df_plot[stream_spec]==1)].drop_duplicates(['freq'], keep='first')
    
    ax.scatter(x='key', y='freq', c = 'r', data= df_nok, 
               s=500, alpha=0.8)
    ax.scatter(x='key', y='freq', c = 'g', data= df_ok, 
               s=500, alpha=0.3)
    ax.set(xlim=arg_dict['x_process_window'], ylim=arg_dict['y_process_window'],
           xlabel='Time[ms]', ylabel='Frequency[Hz]',
           title=plot_title)
    plt.style.use('seaborn-darkgrid')
    plt.rcParams['font.sans-serif'] = 'Arial'
    plt.rcParams['font.family'] = 'sans-serif'
    plt.rcParams['text.color'] = '#909090'
    plt.rcParams['axes.labelcolor']= '#909090'
    plt.rcParams['xtick.color'] = '#909090'
    plt.rcParams['ytick.color'] = '#909090'
    plt.rcParams['font.size']=14
    color_palette_list = ['#009ACD', '#ADD8E6', '#63D1F4', '#0EBFE9',   
                          '#C1F0F6', '#0099CC']
    ax.legend(frameon=False, loc='upper left', ncol=2)
    d_time = datetime.datetime.now().strftime("%y-%m-%d-%H-%M")
    if arg_dict['save']:
        folder = '.\Data\\'
        filename = folder + d_time + '_' + arg_dict['name'] + '_' + stream_spec + '.png'
        plt.savefig(filename)
    plt.show()

#### Check if stream in spec

In [6]:
def check_spec(item, usl, lsl):
    if item < usl and item > lsl:
        return True
    else:
        return False

#### Shift values in selected columns

In [7]:
def shift_values(df, columns,lag):
    '''
    The function returns a new data frame with added new colums with a shifted value
    df: data frame 
    coulmns: a list of columns to be shifted
    lag: how many indexes in row columns will be shifted
    '''
    lags = (lag, lag)
    dfr= df.assign(**{
        '{}(t-{})'.format(col, t): df[col].shift(t)
        for t in lags
        for col in columns
    })
    return dfr

#### Add frequency labels

In [10]:
def do_freq_label(freq):
    if freq/1000000>=1:
        return str(freq/1000000) + "MHz"
    else:
        if freq/1000>=1:
            return str(freq/1000) + "kHz"
        else:
            return str(freq)

In [11]:
import DataReader as dtrd
import FrequencyFinder as freq

## ACT tool and Generator logs import using helper classes in DataReader script.

In [15]:
act_tool_log = "Test_results"
generator_log = "Monitor"

In [16]:
act_tool = dtrd.ActToolLog(act_tool_log)
act_tool.read_act_tool_log()
act_tool.csv_write()

In [17]:
generator = dtrd.GeneratorLog(generator_log)
generator.read_generator_log()
generator.csv_write()

In [18]:
ff = freq.FrequencyFinder(generator.monitor)

### Dictionaries

In [19]:
act_tool.header

['act_time_ms', 'time', 'n_des_gov', 'engine_rpm', 'act_tool_id']

In [20]:
act_tool.act_tool

{'start': 'time', 'stop': '*'}

In [21]:
generator.header[0]

['gen_time_ms', 'time_accu', 'command', 'value', 'monitor_id']

In [22]:
generator.monitor

{'start': 'ms:-> on',
 'stop': 'End:',
 'marker_off': 'to:gen:att',
 'generate': 'to:start',
 'frq_marker': 'to:gen:frq'}

### Read CSV files written previously to \Data\ subfolder

In [23]:
data = pd.read_csv(act_tool.csv_file_path)

In [24]:
data_b = pd.read_csv(generator.csv_file_path)

#### Checking read data

In [25]:
data.head()

,act_time_ms,time,n_des_gov,engine_rpm,act_tool_id
0,0,0.0,3119.3,3125.0,1
1,1,0.1,3138.3,3125.0,2
2,2,0.1,3138.3,3125.0,2
3,3,0.1,3138.3,3125.0,2
4,4,0.1,3138.3,3125.0,2


In [26]:
data.tail()

,act_time_ms,time,n_des_gov,engine_rpm,act_tool_id
9996,9996,10.0,3100.0,3125.0,101
9997,9997,10.0,3100.0,3125.0,101
9998,9998,10.0,3100.0,3125.0,101
9999,9999,10.0,3100.0,3125.0,101
10000,10000,10.0,3100.0,3125.0,101


In [27]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10001 entries, 0 to 10000
Data columns (total 5 columns):
act_time_ms    10001 non-null int64
time           10001 non-null float64
n_des_gov      10001 non-null float64
engine_rpm     10001 non-null float64
act_tool_id    10001 non-null int64
dtypes: float64(3), int64(2)
memory usage: 390.7 KB


In [28]:
data.shape

(10001, 5)

In [29]:
data.describe()

,act_time_ms,time,n_des_gov,engine_rpm,act_tool_id
count,10001.000000,10001.000000,10001.000000,10001.000000,10001.000000
mean,5000.000000,5.049495,2261.000830,2250.087491,51.494951
std,2887.184355,2.887049,920.129503,875.043745,28.870487
min,0.000000,0.000000,1050.900000,1375.000000,1.000000
25%,2500.000000,2.500000,1375.500000,1375.000000,26.000000
50%,5000.000000,5.000000,2407.400000,3125.000000,51.000000
75%,7500.000000,7.500000,3127.800000,3125.000000,76.000000
max,10000.000000,10.000000,4199.100000,3125.000000,101.000000
